In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_stt = pd.read_excel('data/고객입력정보_변환.xlsx', dtype={'세션아이디':str})
df_call = pd.read_excel('data/전체상담내역.xlsx', dtype={'세션 아이디':str})

In [ ]:
df_stt.head()

In [ ]:
df_call.head()

In [ ]:
df_stt.rename(columns={"세션아이디":"session_id", "전화번호":"phone_number"}, inplace=True)
df_call.rename(columns={"세션 아이디":"session_id", "전화번호":"phone_number"}, inplace=True)

In [ ]:
# df_call.sort_values(by='시작시간', ascending=False, inplace=True)
# df_stt['시작시간'] = df_stt.apply(lambda x:df_call.loc[(df_call['전화번호']==x['전화번호']) & (df_call['시작시간']<=x['시간']), '시작시간'].iloc[0], axis=1)

In [ ]:
df = df_stt.merge(df_call, how='left', left_on=['session_id', 'phone_number', '날짜'], right_on=['session_id', 'phone_number', '날짜'])
df.head()

In [ ]:
df_stt.shape

In [ ]:
df.drop_duplicates(inplace=True)
df.shape

In [ ]:
# # 콜 넘버링하기
# df.sort_values(['시간', 'session_id'], inplace=True)
# idarr = df.session_id.unique()
# df["No"] = df.session_id.apply(lambda x:int(np.where(idarr==x)[0])+1)

# df

In [ ]:
# # 발화턴 추가
# df.sort_values(['날짜', '시간', 'session_id'], inplace=True)

# turn = []
# sid_b = 0

# for sid in df.session_id:
#     if sid == sid_b:
#         n += 1
#     else:
#         n = 1
#     turn.append(n)
#     sid_b = sid

# df['발화턴'] = turn
# df

In [ ]:
df = df[['session_id', 'phone_number', '날짜', '시간', '마지막 대화', '상담시간', '통화결과', '대화명', 'STT']]
df

In [ ]:
air_ids = df.loc[df['대화명'].str.contains("에어컨"), "session_id"].unique()
df.loc[df["session_id"].isin(air_ids), "제품"] = "에어컨"

ref_ids = df.loc[df['대화명'].str.contains("냉장고"), "session_id"].unique()
df.loc[df["session_id"].isin(ref_ids), "제품"] = "냉장고"

In [ ]:
df.loc[~df['제품'].isna(), '마지막 대화'].unique()

In [ ]:
lasts = ['R - 진행', '질의 1', '질의 2', '증상 확인', '세부 제품 확인 (김치냉장고)', '세부 제품 확인 (냉장고)', '세부 제품 확인 (에어컨)']
df.loc[(~df['제품'].isna()) & (df['마지막 대화'].isin(lasts)), "분석대상"] = "Y"

In [ ]:
df.fillna("", inplace=True)

In [ ]:
df

In [ ]:
yymmdd = lambda x:x[2:4]+x[5:7]+x[8:10]
df.to_excel(f"result/voicebot_stt_{yymmdd(df['날짜'].min())}~{yymmdd(df['날짜'].max())}.xlsx", index=False)